<a href="https://colab.research.google.com/github/JuliaHdzQ/UFV_VisualizaciondeDatos/blob/main/clase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PASOS PREVIOS A INICIAR UN PROYECTO DE VISUALIZACION DE DATOS**

1. Definir el MENSAJE que queremos transmitir
¿Para quien es la visualización?
Datos de tipos exploratorio

2. OBJETIVO: responder bien a las preguntas propuestas
3. Conocer los datos
4. Conocer la persona o personas a las que nos dirigimos
5. Conocer los limitantes.


* **COMPRENSION DE LOS DATOS**.

1. *Country*: pais al que pertenece el dato
2. *Subrand*: marca
3. *Año* de las ventas reales / de el año sobre el que hago la prediccion
4. *Mes* de las ventas reales / del mes sobre el que hago la prediccion
5. Escenario: puede tener dos valores
  * AI forecast: prediccion realizada para ese mes y ese año (entonces tendremos valores en los siguientes dos campos)
  * Actual: venta real en ese mes y ese año

6. Forecast: en que mes se hizo la prediccion, despues de la p, viene el numero del mes en que se hizo la prediccion mas uno. Es decir AI_P02F es enero
7. Forecast year: el año en que se hizo la predicción
8. Amount: la cantidad

Una vez visto es interesante hacerse alguna preguntas:

Cual es el horizonte de previsión
Que años incluye el estudio
Coinciden los años de valores reales y predicciones.

Explicacion ampliada de forecast:

AI_PF (2023) se hizo a finales de 2022 para enero de 2023

AI_PF02 (2023) estabamos a finales de enero, por lo que seran para febrero ..

**PREGUNTAS QUE HAY QUE RESPONDER:**

1. ¿Como se distribuyen las ventas realizadas?
- Cada pais
- Cada mes y año
- Cada marca

** NO solo usar de distribucion, hay otros que hemos visto que tambien pueden ser buenos aqui.
** Gran parte esta en filtrar bien los datos.

2. ¿Cual es la tendencia y estacionalidad de?
- Todas las ventas del pais con menos ventas
- La marca con mas ventas

3. ¿Cuales son las predicciones hechas en España y como de buenas son ?

## Carga de la base de datos

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [12]:
from google.colab import drive
drive.mount('/content/drive')

# Especicicamos la ruta del archivo CSV en Google Drive
ruta_archivo_drive = '/content/drive/MyDrive/datos_ejercicio_ventas.csv'

# Cargamos el archivo CSV en un DataFrame de pandas
data = pd.read_csv(ruta_archivo_drive)

# Mostramos el Dataframe, verificando que los datos se han cargado correctamente
data


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,7.543562e+05
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,5.600306e+05
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,8.850198e+04
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,3.632245e+05
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,3.961761e+05
...,...,...,...,...,...,...,...,...
18661,Great Britain,Pepsi Regular (L3),2024,2,AI_forecast,AI_P10F,2023.0,1.313511e+06
18662,Hungary,Pepsi Regular (L3),2024,7,AI_forecast,AI_P07F,2023.0,1.314395e+06
18663,Norway,7up (L3),2024,1,AI_forecast,AI_P05F,2023.0,0.000000e+00
18664,Portugal,Lipton (L3),2024,3,AI_forecast,AI_P02F,2023.0,5.330634e+05


## PASOS PREVIOS PARA LA COMPRENSIÓN DE LA BASE DE DATOS

In [14]:
print("Forma del DataFrame:",data.shape)

Forma del DataFrame: (18666, 8)


Tenemos por tanto ocho variables de cada dato, y tenemos un conjunto de 18666 datos.
Vamos a investigar un poco a cerca de cada una de estas variables:

In [16]:
paises = data['COUNTRY'].unique()
print(paises)
print(paises.shape)

['Portugal' 'Great Britain' 'Spain' 'Hungary' 'Norway' 'Denmark'
 'Netherlands' 'Italy' 'Czech']
(9,)


Vamos a hacer lo mismo, pero ahora con las marcas que tenemos

In [17]:
marcas = data['SUBBRAND'].unique()
print(marcas)
print(marcas.shape)

['Lipton (L3)' 'Pepsi Max (L3)' '7up (L3)' 'Pepsi Regular (L3)'
 'Mountain Dew (L3)' '7up Free (L3)']
(6,)


Temos 9 paises distintos, y 6 marcas distintas, vamos a ver con cuantas marcas trabaja cada país, utilizando un grafico de barras o histo

In [20]:
# pip install plotly_express

In [21]:
import plotly_express as px

fig = px.histogram(data, x='COUNTRY', nbins=20, title="Mi histograma")

# Crear el histograma usando Plotly
# Mostrar el histograma
fig.show()

In [23]:
# Contamos las marcas distintas en cada país
grouped_data = data.groupby('COUNTRY')['SUBBRAND'].nunique().reset_index()

# Renombramos la columna para facilitar su uso
grouped_data.columns = ['COUNTRY', 'Unique Brands']

# Y es el número de marcas distintas
fig = px.histogram(grouped_data, x='COUNTRY', y='Unique Brands', nbins=20, title="Número de marcas distintas por país")

# Mostrar el histograma
fig.show()


### Numero de actuals y predicciones
Vamos a ver el numero de actuals (de ventas reales) y el numero de Forecast (de predicciones)

In [29]:
# Comprobamos que solo hay los dos valores de SCENARIO que esperamos y queremos entender.
v = data['SCENARIO'].unique()
print(v)
print(v.shape)

# Imprimimos el numero de ventas reales
num_ventas = (data['SCENARIO'] == 'actual').sum()  # Suma True (1) para 'A'
print('Numero total de ventas reales:', num_ventas)

# Imprimimos el numero de predicciones
num_predicciones = (data['SCENARIO'] == 'AI_forecast').sum()
print('Numero total de predicciones:', num_predicciones)

['AI_forecast' 'actual']
(2,)
Numero total de ventas reales: 900
Numero total de predicciones: 17766


### Horizonte de precisión

Para saberlo, vamos a usar como pais Spain, y como marca: Pepsi Max (L3)

In [34]:
# Definir el país y la bebida específicos, y el mes
pais_especifico = "Spain"
bebida_especifica = "Lipton (L3)"
mes_especifico = "AI_P02F"

horizonte_de_precision = ((data['COUNTRY'] == pais_especifico) &
                   (data['SUBBRAND'] == bebida_especifica) &
                   (data['FORECAST'] == mes_especifico)).sum()

print('Horizonte de precision:', horizonte_de_precision)

Horizonte de precision: 18


vER CUANTOS DATOS REALIES Y CUANTOS PREDICHOS HAY ACTUALS Y FORECAST

HORIZONTE DE PRECISION : numero de puntos que hago la prediccion, si frecuente es un mes, pues 6 son 6 meses (el horizonte es igual para todas las predicciones)

NUMERO DE PAISES Y CUALES HAY

HISTORICO DE DATOS DE FORECAST Y ACTUALS

CUANTOS FORECAST DISTINTOS TENGO


